In [1]:
from itertools import combinations_with_replacement

from ehrhart_quasi_polynomial.ehrhart_piecewise import (
    PiecewiseEhrhartQuasiPolynomial as PEQP,
    create_polytope_from_matrix,
    secondary_fan,
    _process_fan_vectors,
    _compute_change_of_basis_matrices,
    _compute_periods,
    _generate_cone_points)

from ehrhart_quasi_polynomial import *

In [2]:
from sage.calculus.var import var
from sage.functions.other import ceil, factorial
from sage.geometry.cone import Cone
from sage.geometry.polyhedron.constructor import Polyhedron
from sage.matrix.constructor import Matrix
from sage.modules.free_module_element import free_module_element
from sage.modules.free_quadratic_module_integer_symmetric import IntegralLattice
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.rational_field import QQ
from sage.symbolic.ring import SR

In [3]:
ehr = lambda A, b: ehrhart_quasi_polynomial(create_polytope_from_matrix(A, b).Vrepresentation())
num_int_points = lambda A, b: len(create_polytope_from_matrix(A, b).integral_points())

In [4]:
def __call__(self, point):
    if len(point) != self._amb_dim:
            raise ValueError("Dimension of ``point`` needs to be equal to the ambient"
                              f" dimension of ``self`` which is {self._amb_dim}.")
    
    for k, cone_dict in enumerate(self._cone_dicts):
            if point in cone_dict["cone"]:
                off_set = cone_dict["quotient"][cone_dict["quotient"](point)[0]]
                eval_p = ( cone_dict["change_of_basis_inverse"]*(
                    free_module_element(point) - off_set.lift()) )[:self._num_variables]

                result = cone_dict["polynomials"][off_set](*eval_p)
                if isinstance(result, int):
                    return result
                else:
                    return round(result), k, off_set, eval_p
    return 0, None, None, point

PEQP.__call__ = __call__

In [5]:
def test_combinations(p, A, xrange=range(-10, 11)):
        for b in combinations_with_replacement(xrange, A.nrows()):
            expected = num_int_points(A, b)
            actual = p.evaluate(b)
            rounded = p(b)[0]
            if actual != expected:
                print(b, expected, actual, rounded)
        print("all points were tested")

In [6]:
A = Matrix([[-1, 0], [0, -1], [1, 1]])
p = PEQP(A)

In [7]:
p._cone_dicts[0]["polynomials"]

{(0): 9/2*x^2 + 9/2*x + 1, (1): 9/2*x^2 + 3/2*x, (2): 9/2*x^2 - 3/2*x}

In [8]:
test_combinations(p, A)

all points were tested


In [9]:
B = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
q = PEQP(B)

In [10]:
q._cone_dicts[0]["polynomials"]

{(0, 0): 25/2*x1^2 + 15/2*x1 + 1,
 (0, 1): 25/2*x1^2 + 15/2*x1 + 1,
 (0, 2): 25/2*x1^2 + 15/2*x1 + 1,
 (0, 3): 25/2*x1^2 + 15/2*x1 + 1,
 (0, 4): 25/2*x1^2 + 15/2*x1 + 1,
 (1, 0): 25/2*x1^2 - 5/2*x1,
 (1, 1): 25/2*x1^2 - 5/2*x1,
 (1, 2): 25/2*x1^2 - 5/2*x1,
 (1, 3): 25/2*x1^2 - 5/2*x1,
 (1, 4): 25/2*x1^2 - 5/2*x1,
 (2, 0): 25/2*x1^2 - 25/2*x1 + 3,
 (2, 1): 25/2*x1^2 - 25/2*x1 + 3,
 (2, 2): 25/2*x1^2 - 25/2*x1 + 3,
 (2, 3): 25/2*x1^2 - 25/2*x1 + 3,
 (2, 4): 25/2*x1^2 - 25/2*x1 + 3,
 (3, 0): 25/2*x1^2 - 45/2*x1 + 10,
 (3, 1): 25/2*x1^2 - 45/2*x1 + 10,
 (3, 2): 25/2*x1^2 - 45/2*x1 + 10,
 (3, 3): 25/2*x1^2 - 45/2*x1 + 10,
 (3, 4): 25/2*x1^2 - 45/2*x1 + 10,
 (4, 0): 25/2*x1^2 - 65/2*x1 + 21,
 (4, 1): 25/2*x1^2 - 65/2*x1 + 21,
 (4, 2): 25/2*x1^2 - 65/2*x1 + 21,
 (4, 3): 25/2*x1^2 - 65/2*x1 + 21,
 (4, 4): 25/2*x1^2 - 65/2*x1 + 21}

In [11]:
test_combinations(q, B)

all points were tested


In [12]:
C = Matrix([[-1, 0], [0, -1], [1, 2], [0, 1]])
r = PEQP(C)

In [13]:
test_combinations(r, C, range(-6, 7))

(-6, 1, 4, 4) 1 3/4 1
(-6, 1, 4, 6) 1 3/4 1
(-6, 1, 5, 6) 2 9/4 2
(-6, 1, 6, 6) 4 15/4 4
(-6, 2, 2, 3) 1 3/4 1
(-6, 2, 2, 5) 1 3/4 1
(-6, 2, 3, 3) 2 9/4 2
(-6, 2, 3, 5) 2 9/4 2
(-6, 2, 4, 5) 4 15/4 4
(-6, 2, 5, 5) 6 25/4 6
(-6, 3, 3, 4) 6 25/4 6
(-6, 3, 3, 6) 6 25/4 6
(-6, 3, 4, 4) 9 35/4 9
(-6, 3, 4, 6) 9 35/4 9
(-6, 3, 5, 6) 12 49/4 12
(-6, 3, 6, 6) 16 63/4 16
(-6, 4, 4, 5) 16 63/4 16
(-6, 4, 5, 5) 20 81/4 20
(-6, 5, 5, 6) 30 121/4 30
(-6, 5, 6, 6) 36 143/4 36
(-5, 0, 5, 5) 1 3/4 1
(-5, 1, 3, 4) 1 3/4 1
(-5, 1, 3, 6) 1 3/4 1
(-5, 1, 4, 4) 2 9/4 2
(-5, 1, 4, 6) 2 9/4 2
(-5, 1, 5, 6) 4 15/4 4
(-5, 1, 6, 6) 6 25/4 6
(-5, 2, 2, 3) 2 9/4 2
(-5, 2, 2, 5) 2 9/4 2
(-5, 2, 3, 3) 4 15/4 4
(-5, 2, 3, 5) 4 15/4 4
(-5, 2, 4, 5) 6 25/4 6
(-5, 2, 5, 5) 9 35/4 9
(-5, 3, 3, 4) 9 35/4 9
(-5, 3, 3, 6) 9 35/4 9
(-5, 3, 4, 4) 12 49/4 12
(-5, 3, 4, 6) 12 49/4 12
(-5, 3, 5, 6) 16 63/4 16
(-5, 3, 6, 6) 20 81/4 20
(-5, 4, 4, 5) 20 81/4 20
(-5, 4, 5, 5) 25 99/4 25
(-5, 5, 5, 6) 36 143/4 36
(-5, 5, 6, 6) 42 16

In [14]:
#for off_set, poly in list(r._cone_dicts[0]["polynomials"].items()):
#    print(ehr(C, r._rays[2] + off_set.lift()))

In [15]:
D = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
s = PEQP(D)

In [16]:
s._cone_dicts[0]["polynomials"]

{(0, 0, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 4*x1*x2 + 2*x0 + 2*x1 + 2*x2 + 1,
 (0, 0, 1): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 2*x0,
 (0, 1, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x1*x2 + 2*x1,
 (0, 1, 1): 8*x0*x1*x2 + 4*x0*x1,
 (1, 0, 0): 8*x0*x1*x2 + 4*x0*x2 + 4*x1*x2 + 2*x2,
 (1, 0, 1): 8*x0*x1*x2 + 4*x0*x2,
 (1, 1, 0): 8*x0*x1*x2 + 4*x1*x2,
 (1, 1, 1): 8*x0*x1*x2}

In [17]:
test_combinations(s, D, range(-6, 7))

all points were tested


In [18]:
E = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 1, 1]])
t = PEQP(E)

In [19]:
t._cone_dicts[0]["polynomials"]

{(0): 32/3*x^3 + 16*x^2 + 22/3*x + 1,
 (1): 32/3*x^3 + 8*x^2 + 4/3*x,
 (2): 32/3*x^3 - 2/3*x,
 (3): 32/3*x^3 - 8*x^2 + 4/3*x}

In [20]:
test_combinations(t, E, range(-6, 7))

all points were tested


In [21]:
ehr(E, t._rays[0])

QuasiPolynomialElement(Ring of Quasi-Polynomials over Rational Field, [[1], [22/3], [16], [32/3]])

## trying to figure out what the right coordinate transformation is

In [22]:
p._cone_dicts[0]["polynomials"]

{(0): 9/2*x^2 + 9/2*x + 1, (1): 9/2*x^2 + 3/2*x, (2): 9/2*x^2 - 3/2*x}

In [23]:
S = PolynomialRing(QQ, "y", 3)

In [24]:
t = p._cone_dicts[0]["change_of_basis_matrix"]*free_module_element(p._cone_dicts[0]["scaled_rays"][0])*free_module_element(S.gens())
t

2*y0 + 2*y1 - y2

In [27]:
dict = p._cone_dicts[0]
dict["transformed"] = {}
for off_set, poly in dict["polynomials"].items():
    y = dict["change_of_basis_matrix"]*(free_module_element(S.gens(), SR) - free_module_element(off_set.lift(), SR))
    dict["transformed"][off_set] = dict["polynomials"][off_set](y[0])
dict["transformed"]

{(0): 9/2*(y0 + y1)^2 + 9/2*y0 + 9/2*y1 + 1,
 (1): 9/2*(y0 + y1 + 1)^2 + 3/2*y0 + 3/2*y1 + 3/2,
 (2): 9/2*(y0 + y1 + 2)^2 - 3/2*y0 - 3/2*y1 - 3}

In [28]:
dict["transformed"][list(dict["transformed"].keys())[0]](y0=2, y1=2, y2=1)

91

In [32]:
num_int_points(A, (2, 2, 1))

21

In [34]:
p((2, 2, 1))[0]

21

In [29]:
def test_transform(self, point):
    if len(point) != self._amb_dim:
            raise ValueError("Dimension of ``point`` needs to be equal to the ambient"
                              f" dimension of ``self`` which is {self._amb_dim}.")
    
    for k, cone_dict in enumerate(self._cone_dicts):
            if point in cone_dict["cone"]:
                # print(True)
                off_set = cone_dict["quotient"][cone_dict["quotient"](point)[0]]
                return dict["transformed"][off_set](y0=point[0], y1=point[1], y2=point[2])
    return 0

p.test = test_transform

In [30]:
def test(p, A, xrange=range(-10, 11)):
        for b in combinations_with_replacement(xrange, A.nrows()):
            expected = num_int_points(A, b)
            actual = p.test(p, b)
            if actual != expected:
                print(b, expected, actual)
        print("all points were tested")

In [31]:
test(p, A, xrange=range(-5, 6))

(-5, 0, 5) 1 91
(-5, 1, 4) 1 55
(-5, 1, 5) 3 21
(-5, 2, 3) 1 28
(-5, 2, 4) 3 6
(-5, 2, 5) 6 15
(-5, 3, 3) 3 0
(-5, 3, 4) 6 3
(-5, 4, 4) 10 1
(-5, 4, 5) 15 3
(-5, 5, 5) 21 6
(-4, -1, 5) 1 91
(-4, 0, 4) 1 55
(-4, 0, 5) 3 21
(-4, 1, 3) 1 28
(-4, 1, 4) 3 6
(-4, 1, 5) 6 15
(-4, 2, 2) 1 10
(-4, 2, 3) 3 0
(-4, 2, 4) 6 3
(-4, 3, 3) 6 0
(-4, 3, 4) 10 1
(-4, 3, 5) 15 3
(-4, 4, 5) 21 6
(-4, 5, 5) 28 10
(-3, -2, 5) 1 91
(-3, -1, 4) 1 55
(-3, -1, 5) 3 21
(-3, 0, 3) 1 28
(-3, 0, 4) 3 6
(-3, 0, 5) 6 15
(-3, 1, 2) 1 10
(-3, 1, 3) 3 0
(-3, 1, 4) 6 3
(-3, 2, 3) 6 0
(-3, 2, 4) 10 1
(-3, 2, 5) 15 3
(-3, 3, 3) 10 1
(-3, 3, 5) 21 6
(-3, 4, 5) 28 10
(-3, 5, 5) 36 66
(-2, -2, 4) 1 55
(-2, -2, 5) 3 21
(-2, -1, 3) 1 28
(-2, -1, 4) 3 6
(-2, -1, 5) 6 15
(-2, 0, 2) 1 10
(-2, 0, 3) 3 0
(-2, 0, 4) 6 3
(-2, 1, 3) 6 0
(-2, 1, 4) 10 1
(-2, 1, 5) 15 3
(-2, 2, 3) 10 1
(-2, 2, 5) 21 6
(-2, 3, 3) 15 36
(-2, 3, 5) 28 10
(-2, 4, 5) 36 66
(-2, 5, 5) 45 78
(-1, -1, 2) 1 10
(-1, -1, 3) 3 0
(-1, -1, 4) 6 3
(-1, 0, 3) 6 0
(-1, 0,